#### `all.jsonl`

In [ ]:
already_got_id = set()

#real_collection_file = open(collection_file, 'w')
collection_dict = {}
collection_file = "./goldilocks_reproduce/collection/all.jsonl"
collection_lines = []
with open(collection_file) as f:
    for line in tqdm(f):
        data_list = json.loads(line)
        id = data_list["pmid"]
        if id in already_got_id or id == "null" or id is None:
            continue
        else:
            already_got_id.add(id)
            title = data_list["title"]
            content = data_list["abstract"]
            new_content = title + content
            if new_content != "":
                collection_lines.append(json.dumps({id: tokenised_list}) + '\n')
                collection_dict[id] = tokenised_list
#real_collection_file.writelines(collection_lines)
#real_collection_file.close()
collection_lines.clear()

454314it [04:10, 1815.44it/s]


In [99]:
len(already_got_id)

198110

In [85]:
len(collection_dict.keys())

454273

In [88]:
collection_dict['18485144']

In [2]:
!head -1 ./goldilocks_reproduce/collection/all.jsonl 

{"pmid": "18485144", "title": "Current and future therapy in Alzheimer's disease.", "abstract": "Dementia is increasingly being recognized as one of the most important medical problems in the elderly. As most pharmacological research within the field of dementia is focused on Alzheimer's dementia (AD), this review will focus on pharmacological interventions in AD. Most disease-modifying therapies are based on the amyloid hypothesis. In this hypothesis, the pathological accumulation of Abeta in the brain leads to oxidative stress, neuronal destruction and finally the clinical syndrome of AD. Following this hypothesis, secondary prevention of AD can be made by: decreasing the production of Abeta, stimulation of clearance of Abeta formed or prevention of aggregation of Abeta into amyloid plaques. First a short overview on current approved therapies for AD is given. The main part of  the review will focus on potential disease-modifying therapies for AD that are currently being studied in p

#### Clean `all.jsonl` --> `all_clean.jsonl`

In [127]:
clean_collection_file = open("./goldilocks_reproduce/collection/all_clean.jsonl", 'w')
#clean_collection_dict = {}
already_got_id = set()
collection_file = "./goldilocks_reproduce/collection/all.jsonl"
clean_collection_lines = []
n_repeat = []
n_nan = 0

with open(collection_file) as f:
    for line in tqdm(f):
        data_list = json.loads(line)
        pmid = data_list["pmid"]
        if pmid in already_got_id:
            n_repeat.append(pmid)
            continue
        elif pmid is None:
            n_nan += 1
            continue
        else:
            already_got_id.add(pmid)
            title = data_list["title"]
            abstract = data_list["abstract"]
            content = title + abstract
            if content != "":
                clean_collection_lines.append(json.dumps({"pmid": pmid, "title": title, "abstract": abstract}) +'\n')
                #clean_collection_dict[pmid] = content
clean_collection_file.writelines(clean_collection_lines)
clean_collection_file.close()

454314it [00:03, 115421.38it/s]


In [128]:
len(n_repeat) 

34

In [15]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [16]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'This': 3,
             'is': 2,
             'the': 1,
             'Hugging': 1,
             'Face': 1,
             'Course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

In [17]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


In [18]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [19]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [20]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [21]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('T', '##h'): 0.125
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('t', '##h'): 0.03571428571428571
('##h', '##e'): 0.011904761904761904


In [22]:
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


In [23]:
vocab.append("ab")

In [24]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [25]:
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

In [26]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [27]:
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', 'ch', '##hm', 'cha', 'chap', 'chapt', '##thm', 'Hu', 'Hug', 'Hugg', 'sh', 'th', 'is', '##thms', '##za', '##zat', '##ut']


In [28]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [29]:
print(encode_word("Hugging"))
print(encode_word("HOgging"))

['Hugg', '##i', '##n', '##g']
['[UNK]']


In [30]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [31]:
tokenize("This is the Hugging Face course!")

['Th',
 '##i',
 '##s',
 'is',
 'th',
 '##e',
 'Hugg',
 '##i',
 '##n',
 '##g',
 'Fac',
 '##e',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']